# NLP (comentários página do Facebook)
Este notebook contém o código referente ao modelo que analisa sentimentos em formato textual. A proposta aqui foi a de criar um modelo que pudesse analisar os comentários públicos da página do Facebook da Passos Mágicos e assim demonstrar o quão impactante suas ações são nas vidas das pessoas. Foram considerados 3 categorias de comentários: positivo, negativo e neutro.

Aqui vale notar que além dos comentários da página do Facebook da ONG, também foram gerados batches de comentários pelo ChatGPT da OpenAI. Isso se fez necessário, pois não haviam muitos comentários ou reviews da ONG que pudessem ser utilizados para treinar efetivamente um NLP.

Os comentários e reviews originais da página da ONG foram misturados juntos dos comentários gerados pelo ChatGPT.

In [1]:
!pip install kaggle

In [2]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 19.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [66]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import unicodedata
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import svm
import joblib
import os
from google.colab import userdata

In [4]:
# vars de ambiente para o Kaggle (aqui também seria possível utilizar o Drive do Google como repositório para a chave)
# o único ponto é que utilizando vars de ambiente, preciso obrigatoriamente utilizar o pacote 'os'
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

pd.set_option('display.max_columns', None)

In [5]:
# Kaggle pt-BR dataset
os.system('mkdir /content/kaggle-data')
os.system('kaggle datasets download -d augustop/portuguese-tweets-for-sentiment-analysis -p /content/kaggle-data')
os.system('unzip /content/kaggle-data/portuguese-tweets-for-sentiment-analysis.zip -d /content/kaggle-data')

0

In [6]:
# NTLK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
print(spacy.util.get_installed_models())

['pt_core_news_sm', 'en_core_web_sm']


In [8]:
nlp_spacy_ptbr = spacy.load("pt_core_news_sm")

## Funções utilitárias para limpeza e normalização de strings

In [9]:
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def tokenizer(text):
    stop_words = stopwords.words("portuguese")
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

def lemmatize(text):
    doc = nlp_spacy_ptbr(text)
    return " ".join([token.lemma_ for token in doc])

## Carregando os comentários e reviews da página do Facebook da ONG Passos Mágicos + comentários gerados pelo ChatGPT da OpenAI

In [105]:
df_1 = pd.read_csv('/content/chatgpt-base-comentarios-negativos.csv', sep=';')
df_2 = pd.read_csv('/content/chatgpt-base-comentarios-neutros.csv', sep=';')
df_3 = pd.read_csv('/content/chatgpt-base-comentarios-positivos.csv', sep=';')
df_4 = pd.read_csv('/content/facebook-mentions.csv', sep=';')
df_5 = pd.read_csv('/content/facebook-reviews.csv', sep=';')
df_ptbr_sentiment_1 = pd.read_csv('/content/kaggle-data/TweetsNeutralHash.csv')
df_ptbr_sentiment_2 = pd.read_csv('/content/kaggle-data/TweetsNeutralNews.csv')
df_ptbr_sentiment_3 = pd.read_csv('/content/kaggle-data/TweetsWithTheme.csv')
df_ptbr_sentiment_4 = pd.read_csv('/content/kaggle-data/NoThemeTweets.csv')

In [106]:
df_ptbr_sentiment_1['categoria'] = df_ptbr_sentiment_1['sentiment'].str.lower()
df_ptbr_sentiment_1['texto'] = df_ptbr_sentiment_1['tweet_text']
df_ptbr_sentiment_1_final = df_ptbr_sentiment_1[['texto', 'categoria']]
df_ptbr_sentiment_1_final = df_ptbr_sentiment_1_final.sample(8000, random_state=777)

In [107]:
df_ptbr_sentiment_2['categoria'] = df_ptbr_sentiment_2['sentiment'].str.lower()
df_ptbr_sentiment_2['texto'] = df_ptbr_sentiment_2['tweet_text']
df_ptbr_sentiment_2_final = df_ptbr_sentiment_2[['texto', 'categoria']]
df_ptbr_sentiment_2_final = df_ptbr_sentiment_2_final.sample(8000, random_state=777)

In [108]:
df_ptbr_sentiment_3['categoria'] = df_ptbr_sentiment_3['sentiment'].str.lower()
df_ptbr_sentiment_3['texto'] = df_ptbr_sentiment_3['tweet_text']
df_ptbr_sentiment_3_final = df_ptbr_sentiment_3[['texto', 'categoria']]
df_ptbr_sentiment_3_final = df_ptbr_sentiment_3_final.sample(8000, random_state=777)

In [109]:
df_ptbr_sentiment_4['categoria'] = df_ptbr_sentiment_4['sentiment'].str.lower()
df_ptbr_sentiment_4['texto'] = df_ptbr_sentiment_4['tweet_text']
df_ptbr_sentiment_4_final = df_ptbr_sentiment_4[['texto', 'categoria']]
df_ptbr_sentiment_4_final = df_ptbr_sentiment_4_final.groupby('categoria', group_keys=False).apply(lambda x: x.sample(min(len(x), 20000), random_state=777))

In [110]:
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_ptbr_sentiment_1_final, df_ptbr_sentiment_2_final, df_ptbr_sentiment_3_final, df_ptbr_sentiment_4_final], ignore_index=True)
df

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo
...,...,...
45055,@viquemf quem é bia e porque vc falaria com el...,positivo
45056,@Sun_diffbreed @WeltonLSutil Sempre! Costumamo...,positivo
45057,além do utero a minha cabeça tmb vai explodir :),positivo
45058,Menino tu fica muito gostoso de barba (stalkee...,positivo


In [111]:
df['categoria'].unique()

array(['negativo', 'neutro', 'positivo'], dtype=object)

In [112]:
df['categoria'].value_counts()

categoria
neutro      16443
positivo    14540
negativo    14077
Name: count, dtype: int64

In [113]:
df.shape

(45060, 2)

In [114]:
df.head()

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo


In [115]:
df = shuffle(df)
df = df.reset_index(drop=True)

In [116]:
df.head()

,texto,categoria
0,"@idetefoder @leandreiss Aí tantassss , o que v...",positivo
1,"sim kkkkkkkk, vivem em uma grande bolha. Quem ...",positivo
2,Igualdade de oportunidade! Esse é a caminho pa...,neutro
3,queria estar conversando com a vivi :(,negativo
4,@099CHK mas você não conversa comigo :(,negativo


In [117]:
df['texto_processado'] = df['texto'].apply(tokenizer)
# df['texto_processado_lemm'] = df['texto_processado'].apply(lemmatize)
df.head()

,texto,categoria,texto_processado
0,"@idetefoder @leandreiss Aí tantassss , o que v...",positivo,idetefoder leandreiss tantassss vale vivemos p...
1,"sim kkkkkkkk, vivem em uma grande bolha. Quem ...",positivo,sim kkkkkkkk vivem grande bolha vota haddad na...
2,Igualdade de oportunidade! Esse é a caminho pa...,neutro,igualdade oportunidade caminho jovens humildes...
3,queria estar conversando com a vivi :(,negativo,queria conversando vivi
4,@099CHK mas você não conversa comigo :(,negativo,chk voce nao conversa comigo


## Dividindo o dataset em treino e teste

In [118]:
X = df['texto_processado'].values
y = df['categoria'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777, stratify=y)

In [119]:
print('X_train:', X_train.size)
print('X_test:', X_test.size)
print('y_train:', y_train.size)
print('y_test:', y_test.size)

X_train: 36048
X_test: 9012
y_train: 36048
y_test: 9012


In [120]:
vect = CountVectorizer(lowercase=False)
vect.fit(X_train)

CountVectorizer(lowercase=False)

In [121]:
X_train = vect.transform(X_train)
X_train

<36048x71341 sparse matrix of type '<class 'numpy.int64'>'
	with 362166 stored elements in Compressed Sparse Row format>

In [122]:
X_test = vect.transform(X_test)
X_test

<9012x71341 sparse matrix of type '<class 'numpy.int64'>'
	with 78063 stored elements in Compressed Sparse Row format>

## Utilização do modelo SVM para categorizar as frases

In [123]:
modelo = svm.SVC(kernel='linear')
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

In [124]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7924988903683977


In [125]:
y_pred_class = [x for x in y_pred]
y_test_class = y_test

print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

    negativo       0.70      0.74      0.72      2815
      neutro       0.96      0.94      0.95      3289
    positivo       0.70      0.67      0.69      2908

    accuracy                           0.79      9012
   macro avg       0.79      0.79      0.79      9012
weighted avg       0.79      0.79      0.79      9012



## Teste do modelo

In [126]:
texto_teste = tokenizer('Não gostei da ONG, não prestaram o serviço de forma adequada.')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)

In [127]:
texto_teste = tokenizer('Gostei da ONG!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['negativo'], dtype=object)

In [128]:
texto_teste = tokenizer('A OGN poderia oferecer mais serviços, mas ok.')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['positivo'], dtype=object)

In [129]:
texto_teste = tokenizer('Parece uma boa ONG, contudo poderia ser mais transparente!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)

## Exportando o modelo para uso no Streamlit

In [ ]:
arquivo_modelo = "model.pkl"
arquivo_vect = "vect.pkl"

joblib.dump(modelo, arquivo_modelo)
joblib.dump(vect, arquivo_vect)

## Importando o modelo para teste

In [ ]:
modelo = joblib.load(arquivo_modelo)
vect = joblib.load(arquivo_vect)

In [ ]:
texto_teste = tokenizer('Parece uma boa ONG, contudo poderia ser mais transparente!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

In [ ]:
texto_teste = tokenizer('ONG maravilhosa')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

# Utilizando NLP para identificar o teor dos comentários dos professores e pedagogos da Passos Mágicos

In [ ]:
df = pd.read_csv('/content/processado_base_full.csv', sep=';')
df.head()

In [ ]:
df_comentarios_destaque_ieg = df[df['DESTAQUE_IEG'].notna()]['DESTAQUE_IEG'].unique()
df_comentarios_destaque_ieg

In [ ]:
df_comentarios_destaque_ida = df[df['DESTAQUE_IDA'].notna()]['DESTAQUE_IDA'].unique()
df_comentarios_destaque_ida

In [ ]:
df_comentarios_destaque_ipv = df[df['DESTAQUE_IPV'].notna()]['DESTAQUE_IPV'].unique()
df_comentarios_destaque_ipv

In [ ]:
df_comentarios_rec_equipe1 = df[df['REC_EQUIPE_1'].notna()]['REC_EQUIPE_1'].unique()
df_comentarios_rec_equipe1

In [ ]:
df_comentarios_rec_equipe2 = df[df['REC_EQUIPE_2'].notna()]['REC_EQUIPE_2'].unique()
df_comentarios_rec_equipe2

In [ ]:
df_comentarios_rec_equipe3 = df[df['REC_EQUIPE_3'].notna()]['REC_EQUIPE_3'].unique()
df_comentarios_rec_equipe3

In [ ]:
df_comentarios_rec_equipe4 = df[df['REC_EQUIPE_4'].notna()]['REC_EQUIPE_4'].unique()
df_comentarios_rec_equipe4

In [ ]:
df_comentarios_rec_ava1 = df[df['REC_AVA_1'].notna()]['REC_AVA_1'].unique()
df_comentarios_rec_ava1

In [ ]:
df_comentarios_rec_ava2 = df[df['REC_AVA_2'].notna()]['REC_AVA_2'].unique()
df_comentarios_rec_ava2

In [ ]:
df_comentarios_rec_ava3 = df[df['REC_AVA_3'].notna()]['REC_AVA_3'].unique()
df_comentarios_rec_ava3

In [ ]:
df_comentarios_rec_ava4 = df[df['REC_AVA_4'].notna()]['REC_AVA_4'].unique()
df_comentarios_rec_ava4

In [ ]:
array_nlp_processamento = np.concatenate((df_comentarios_destaque_ieg, df_comentarios_destaque_ida, df_comentarios_destaque_ipv, df_comentarios_rec_equipe1, df_comentarios_rec_equipe2, df_comentarios_rec_equipe3, df_comentarios_rec_equipe4, df_comentarios_rec_ava1, df_comentarios_rec_ava2, df_comentarios_rec_ava3, df_comentarios_rec_ava4))
array_nlp_processamento

In [ ]:
df_validacao_nlp = pd.DataFrame({
    'texto': array_nlp_processamento
})
df_validacao_nlp['tokenizado'] = df_validacao_nlp['texto'].apply(tokenizer)
df_validacao_nlp.head()

In [ ]:
df_validacao_nlp['previsao'] = df_validacao_nlp['tokenizado'].apply(lambda x: modelo.predict(vect.transform([x])))
df_validacao_nlp.head(50)

In [ ]:
from transformers import pipeline

sentiment_pipeline = pipeline('sentiment-analysis')
result = sentiment_pipeline("Eu não gosto este café!")

print(result)

In [ ]:
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")
classifier(["Eduardo é um homem maravilhoso!","Singas é um gênio!", "Silco é um nojento!"])

In [ ]:
classifier(['Eu gosto de café', 'Eu não gosto de café', 'não gostei muito, mas ok'])

In [ ]:
sentiment_pipeline(['Eu gosto de café', 'Eu não gosto de café', 'não gostei muito, mas ok'])